In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import VOCDetection
import torchvision.transforms as transforms
import numpy as np
import os
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageDraw
from torchvision.transforms.functional import to_pil_image

# Configuration des graphiques matplotlib pour qu'ils s'affichent correctement
import matplotlib
matplotlib.use('TkAgg')  # ou 'Agg' pour éviter les fenêtres graphiques



: 

In [ ]:



# Définition des classes PASCAL VOC
VOC_CLASSES = [
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]

# Configuration
BATCH_SIZE = 2           # Taille du lot
LEARNING_RATE = 0.001    # Taux d'apprentissage initiale
WEIGHT_DECAY = 0.0005    # Régularisation pour éviter le surapprentissage
EPOCHS = 10              # Nombre de passages complets sur le dataset
NUM_CLASSES = 20         # Classes dans PASCAL VOC (sans le fond)
IMG_SIZE = 320        # Résolution d'entrée (carrée) pour YOLO
CHECKPOINT_DIR = "./checkpoints"  # Dossier pour sauvegarder les modèles
MAX_SAMPLES = 3     # Nombre d'échantillons pour le dataset

# Anchors pour les trois échelles
ANCHORS = [
    [(116, 90), (156, 198), (373, 326)],  # Grand
    [(30, 61), (62, 45), (59, 119)],      # Moyen
    [(10, 13), (16, 30), (33, 23)]        # Petit
]

###########################################
# PARTIE 1: ARCHITECTURE DU MODÈLE YOLOv3 #
###########################################

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.leaky = nn.LeakyReLU(0.1)
        
    def forward(self, x):
        return self.leaky(self.bn(self.conv(x)))

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = ConvBlock(channels, channels//2, kernel_size=1)
        self.conv2 = ConvBlock(channels//2, channels, kernel_size=3, padding=1)
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        out += residual
        return out

class Darknet53(nn.Module):
    def __init__(self, block, num_classes=1000):
        super(Darknet53, self).__init__()
        self.num_classes = num_classes
        self.conv1 = ConvBlock(3, 32, kernel_size=3, padding=1)
        self.conv2 = ConvBlock(32, 64, kernel_size=3, stride=2, padding=1)
        
        # Residual blocks
        self.res_block1 = self._make_layer(block, 64, 1)
        self.conv3 = ConvBlock(64, 128, kernel_size=3, stride=2, padding=1)
        
        self.res_block2 = self._make_layer(block, 128, 2)
        self.conv4 = ConvBlock(128, 256, kernel_size=3, stride=2, padding=1)
        
        self.res_block3 = self._make_layer(block, 256, 8)
        self.conv5 = ConvBlock(256, 512, kernel_size=3, stride=2, padding=1)
        
        self.res_block4 = self._make_layer(block, 512, 8)
        self.conv6 = ConvBlock(512, 1024, kernel_size=3, stride=2, padding=1)
        
        self.res_block5 = self._make_layer(block, 1024, 4)
        
        # Pour la classification (si nécessaire)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(1024, num_classes)
        
        # Sorties pour la détection (à adapter selon les besoins)
        self.features = [self.res_block3, self.res_block4, self.res_block5]
    
    def _make_layer(self, block, channels, num_blocks):
        layers = []
        for _ in range(num_blocks):
            layers.append(block(channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        
        x = self.res_block1(x)
        x = self.conv3(x)
        
        x = self.res_block2(x)
        x = self.conv4(x)
        
        route1 = self.res_block3(x)
        x = self.conv5(route1)
        
        route2 = self.res_block4(x)
        x = self.conv6(route2)
        
        route3 = self.res_block5(x)
        
        # Pour la détection, nous retournons les features à différentes échelles
        return route1, route2, route3

class YOLOv3(nn.Module):
    def __init__(self, num_classes=80):
        super(YOLOv3, self).__init__()
        self.num_classes = num_classes
        
        # Backbone
        self.darknet = Darknet53(ResidualBlock)
        
        # Couches de détection
        # Première échelle (grande)
        self.conv_large = nn.Sequential(
            ConvBlock(1024, 512, kernel_size=1),
            ConvBlock(512, 1024, kernel_size=3, padding=1),
            ConvBlock(1024, 512, kernel_size=1),
            ConvBlock(512, 1024, kernel_size=3, padding=1),
            ConvBlock(1024, 512, kernel_size=1)
        )
        self.detect_large = nn.Sequential(
            ConvBlock(512, 1024, kernel_size=3, padding=1),
            nn.Conv2d(1024, 3 * (5 + num_classes), kernel_size=1)  # 3 anchors, 5 pour box (x,y,w,h,conf) + classes
        )
        
        # Upsampling et concaténation
        self.conv_up1 = ConvBlock(512, 256, kernel_size=1)
        self.upsample1 = nn.Upsample(scale_factor=2, mode='nearest')
        
        # Deuxième échelle (moyenne)
        self.conv_medium = nn.Sequential(
            ConvBlock(768, 256, kernel_size=1),  # 256 (upsampled) + 512 (route2)
            ConvBlock(256, 512, kernel_size=3, padding=1),
            ConvBlock(512, 256, kernel_size=1),
            ConvBlock(256, 512, kernel_size=3, padding=1),
            ConvBlock(512, 256, kernel_size=1)
        )
        self.detect_medium = nn.Sequential(
            ConvBlock(256, 512, kernel_size=3, padding=1),
            nn.Conv2d(512, 3 * (5 + num_classes), kernel_size=1)
        )
        
        # Upsampling et concaténation
        self.conv_up2 = ConvBlock(256, 128, kernel_size=1)
        self.upsample2 = nn.Upsample(scale_factor=2, mode='nearest')
        
        # Troisième échelle (petite)
        self.conv_small = nn.Sequential(
            ConvBlock(384, 128, kernel_size=1),  # 128 (upsampled) + 256 (route1)
            ConvBlock(128, 256, kernel_size=3, padding=1),
            ConvBlock(256, 128, kernel_size=1),
            ConvBlock(128, 256, kernel_size=3, padding=1),
            ConvBlock(256, 128, kernel_size=1)
        )
        self.detect_small = nn.Sequential(
            ConvBlock(128, 256, kernel_size=3, padding=1),
            nn.Conv2d(256, 3 * (5 + num_classes), kernel_size=1)
        )
    
    def forward(self, x):
        if isinstance(x, list):
            x = torch.stack(x)
        
        # Obtenir les features du backbone
        route1, route2, route3 = self.darknet(x)
        
        # Première échelle (grande - détecte les grands objets)
        large = self.conv_large(route3)
        detect_large = self.detect_large(large)
        
        # Upsampling et concaténation avec route2
        up1 = self.conv_up1(large)
        up1 = self.upsample1(up1)
        medium_in = torch.cat([up1, route2], dim=1)
        
        # Deuxième échelle (moyenne - détecte les objets de taille moyenne)
        medium = self.conv_medium(medium_in)
        detect_medium = self.detect_medium(medium)
        
        # Upsampling et concaténation avec route1
        up2 = self.conv_up2(medium)
        up2 = self.upsample2(up2)
        small_in = torch.cat([up2, route1], dim=1)
        
        # Troisième échelle (petite - détecte les petits objets)
        small = self.conv_small(small_in)
        detect_small = self.detect_small(small)
        
        return detect_large, detect_medium, detect_small

###########################################
# PARTIE 2: CHARGEMENT ET PRÉPARATION DES DONNÉES #
###########################################

class VOCSubset(Dataset):
    def __init__(self, root="./data", year="2012", image_set="train", download=True, transform=None, target_transform=None, max_samples=100):
        """
        Classe pour charger et traiter un sous-ensemble de PASCAL VOC
        
        Args:
            root: Répertoire racine des données
            year: Année du dataset PASCAL VOC ('2007' ou '2012')
            image_set: Ensemble d'images ('train', 'val', 'test')
            download: Télécharger automatiquement si non présent
            transform: Transformations à appliquer sur les images
            target_transform: Transformations à appliquer sur les annotations
            max_samples: Nombre maximum d'échantillons à inclure
        """
        self.voc = VOCDetection(root=root, year=year, image_set=image_set, download=download)
        self.transform = transform
        self.target_transform = target_transform
        
        # Sélectionner un sous-ensemble aléatoire
        total_samples = len(self.voc)
        self.indices = random.sample(range(total_samples), min(max_samples, total_samples))
        
        # Créer un mapping des classes vers des indices
        self.class_to_idx = {cls: i for i, cls in enumerate(VOC_CLASSES)}
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        voc_idx = self.indices[idx]
        img, target = self.voc[voc_idx]
        
        # Extraction des boîtes et classes des annotations VOC
        boxes = []
        labels = []
        
        for obj in target['annotation']['object']:
            # Extraire les coordonnées de la boîte
            bbox = obj['bndbox']
            xmin = float(bbox['xmin'])
            ymin = float(bbox['ymin'])
            xmax = float(bbox['xmax'])
            ymax = float(bbox['ymax'])
            
            # Extraire la classe
            class_name = obj['name']
            class_idx = self.class_to_idx[class_name]
            
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(class_idx)
        
        # Conversion en tenseurs PyTorch
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        # Créer un dictionnaire de cibles au format attendu par PyTorch
        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([voc_idx]),
            'area': (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0]),
            'iscrowd': torch.zeros((len(boxes),), dtype=torch.int64)
        }
        
        # Appliquer les transformations si spécifiées
        if self.transform:
            img = self.transform(img)
        if self.target_transform:
            target = self.target_transform(target)
            
        return img, target

def collate_fn(batch):
    """
    Fonction personnalisée pour regrouper les échantillons en lots
    """
    images = []
    targets = []
    for img, tgt in batch:
        images.append(img)
        targets.append(tgt)
    return images, targets

def prepare_data_loaders(batch_size=4, max_samples=100):
    """
    Prépare les DataLoaders pour l'entraînement et la validation
    """
    # Transformation standard pour les images
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((416, 416)),  # Redimensionner pour YOLOv3
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Préparer les jeux de données
    train_dataset = VOCSubset(
        root="./data", 
        year="2012", 
        image_set="train", 
        download=True, 
        transform=transform, 
        max_samples=max_samples
    )
    
    val_dataset = VOCSubset(
        root="./data", 
        year="2012", 
        image_set="val", 
        download=True, 
        transform=transform, 
        max_samples=max_samples // 2  # Moitié moins d'échantillons pour validation
    )
    
    # Créer les DataLoaders
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        collate_fn=collate_fn,
        num_workers=0
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        collate_fn=collate_fn,
        num_workers=2
    )
    
    return train_loader, val_loader

###########################################
# PARTIE 3: FONCTIONS DE POST-TRAITEMENT #
###########################################

def transform_predictions(predictions, input_size, anchors, num_classes, device):
    """
    Transforme les prédictions brutes du réseau en coordonnées de boîtes de délimitation.
    """
    batch_size = predictions.size(0)
    grid_size = predictions.size(2)
    
    # Nombre d'attributs par boîte: 5 (x, y, w, h, conf) + num_classes
    stride = input_size // grid_size
    bbox_attrs = 5 + num_classes
    num_anchors = len(anchors)
    
    # Reshape des prédictions
    predictions = predictions.view(batch_size, num_anchors, bbox_attrs, grid_size, grid_size)
    predictions = predictions.permute(0, 1, 3, 4, 2).contiguous()
    
    # Appliquer la fonction sigmoïde pour les coordonnées x, y et la confiance
    predictions[..., 0] = torch.sigmoid(predictions[..., 0])  # x
    predictions[..., 1] = torch.sigmoid(predictions[..., 1])  # y
    predictions[..., 4] = torch.sigmoid(predictions[..., 4])  # conf
    
    # Appliquer sigmoid aux scores de classe
    predictions[..., 5:] = torch.sigmoid(predictions[..., 5:])
    
    # Ajouter les offsets de grille
    grid_x = torch.arange(grid_size).repeat(grid_size, 1).view([1, 1, grid_size, grid_size]).to(device)
    grid_y = torch.arange(grid_size).repeat(grid_size, 1).t().view([1, 1, grid_size, grid_size]).to(device)
    
    scaled_anchors = torch.FloatTensor([(a[0]/stride, a[1]/stride) for a in anchors]).to(device)
    anchor_w = scaled_anchors[:, 0:1].view((1, num_anchors, 1, 1))
    anchor_h = scaled_anchors[:, 1:2].view((1, num_anchors, 1, 1))
    
    # Appliquer les transformations
    predictions[..., 0] += grid_x
    predictions[..., 1] += grid_y
    predictions[..., 2] = torch.exp(predictions[..., 2]) * anchor_w
    predictions[..., 3] = torch.exp(predictions[..., 3]) * anchor_h
    
    # Mettre à l'échelle pour la taille d'entrée
    predictions[..., :4] *= stride
    
    # Reshape pour la suppression non-maximale - IMPORTANT pour corriger l'erreur
    predictions = predictions.view(batch_size, -1, 5 + num_classes)
    
    return predictions

def non_max_suppression(prediction, num_classes, conf_threshold=0.5, nms_threshold=0.4):
    """
    Applique la suppression non-maximale (NMS) aux boîtes prédites.
    """
    # Depuis (centre x, centre y, largeur, hauteur) vers (x1, y1, x2, y2)
    box_corner = prediction.new(prediction.shape)
    box_corner[..., 0] = prediction[..., 0] - prediction[..., 2] / 2
    box_corner[..., 1] = prediction[..., 1] - prediction[..., 3] / 2
    box_corner[..., 2] = prediction[..., 0] + prediction[..., 2] / 2
    box_corner[..., 3] = prediction[..., 1] + prediction[..., 3] / 2
    prediction[..., :4] = box_corner[..., :4]
    
    output = [None for _ in range(len(prediction))]
    
    for image_i, image_pred in enumerate(prediction):
        # Filtrer les boîtes avec un score de confiance inférieur au seuil
        conf_mask = (image_pred[:, 4] >= conf_threshold).squeeze()
        image_pred = image_pred[conf_mask]
        
        # Si aucune boîte ne reste après le filtrage
        if not image_pred.size(0):
            continue
            
        # Obtenir les scores de classe avec la confiance de détection
        class_conf, class_pred = torch.max(image_pred[:, 5:5 + num_classes], 1, keepdim=True)
        
        # Concaténer les scores avec les coordonnées de la boîte
        detections = torch.cat((image_pred[:, :5], class_conf.float(), class_pred.float()), 1)
        
        # Obtenir les classes uniques
        unique_classes = detections[:, -1].cpu().unique()
        
        if prediction.is_cuda:
            unique_classes = unique_classes.cuda()
            detections = detections.cuda()
            
        for c in unique_classes:
            # Obtenir les détections avec cette classe
            detections_class = detections[detections[:, -1] == c]
            
            # Trier par confiance décroissante
            _, conf_sort_index = torch.sort(detections_class[:, 4], descending=True)
            detections_class = detections_class[conf_sort_index]
            
            # Appliquer NMS
            max_detections = []
            while detections_class.size(0):
                # Obtenir la détection avec la confiance la plus élevée
                max_detections.append(detections_class[0].unsqueeze(0))
                
                # Arrêter si nous n'avons plus de détections
                if len(detections_class) == 1:
                    break
                    
                # Obtenir les IOUs pour toutes les autres détections
                ious = bbox_iou(max_detections[-1], detections_class[1:])
                
                # Supprimer les détections avec un IOU supérieur au seuil
                detections_class = detections_class[1:][ious < nms_threshold]
                
            max_detections = torch.cat(max_detections).data
            
            # Ajouter les détections de cette classe au résultat final
            output[image_i] = max_detections if output[image_i] is None else torch.cat((output[image_i], max_detections))
    
    return output

def bbox_iou(box1, box2):
    """
    Calcule l'IOU entre deux ensembles de boîtes.
    """
    # Obtenir les coordonnées des boîtes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1[:, 0], box1[:, 1], box1[:, 2], box1[:, 3]
    b2_x1, b2_y1, b2_x2, b2_y2 = box2[:, 0], box2[:, 1], box2[:, 2], box2[:, 3]
    
    # Obtenir les coordonnées de l'intersection
    inter_rect_x1 = torch.max(b1_x1, b2_x1)
    inter_rect_y1 = torch.max(b1_y1, b2_y1)
    inter_rect_x2 = torch.min(b1_x2, b2_x2)
    inter_rect_y2 = torch.min(b1_y2, b2_y2)
    
    # Calculer l'aire de l'intersection
    inter_area = torch.clamp(inter_rect_x2 - inter_rect_x1 + 1, min=0) * \
                 torch.clamp(inter_rect_y2 - inter_rect_y1 + 1, min=0)
                 
    # Calculer l'aire des deux boîtes
    b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)
    
    # Calculer l'IOU
    iou = inter_area / (b1_area + b2_area - inter_area + 1e-16)
    
    return iou

###########################################
# PARTIE 4: FONCTION DE PERTE ET ENTRAÎNEMENT #
###########################################

class YOLOLoss(nn.Module):
    def __init__(self, anchors, num_classes, img_size=320):
        super(YOLOLoss, self).__init__()
        self.anchors = anchors
        self.num_classes = num_classes
        self.img_size = img_size
        self.mse = nn.MSELoss(reduction="sum")
        self.bce = nn.BCEWithLogitsLoss(reduction="sum")
        self.entropy = nn.CrossEntropyLoss(reduction="sum")
        self.sigmoid = nn.Sigmoid()
        
        # Constants
        self.lambda_coord = 5
        self.lambda_noobj = 0.5
        
    def forward(self, predictions, targets, anchors):
        """
        Calcule la perte pour une sortie YOLO à une échelle donnée
        
        Args:
            predictions: Sortie du réseau à une échelle [B, 3*(5+C), H, W]
            targets: Annotations au format YOLO [B, max_objects, 5+C]
            anchors: Anchors pour cette échelle
        """
        # Récupérer les dimensions
        batch_size = predictions.shape[0]
        grid_size = predictions.shape[2]
        stride = self.img_size // grid_size
        
        # Reshape des prédictions
        predictions = predictions.view(batch_size, 3, 5 + self.num_classes, grid_size, grid_size)
        predictions = predictions.permute(0, 1, 3, 4, 2).contiguous()
        
        # Appliquer sigmoid aux coordonnées x, y et à la confiance
        x = self.sigmoid(predictions[..., 0])
        y = self.sigmoid(predictions[..., 1])
        w = predictions[..., 2]
        h = predictions[..., 3]
        conf = self.sigmoid(predictions[..., 4])
        pred_cls = self.sigmoid(predictions[..., 5:])
        
        # Préparer les masques et les tenseurs pour les cibles
        obj_mask = torch.zeros(batch_size, 3, grid_size, grid_size, dtype=torch.bool, device=predictions.device)
        noobj_mask = torch.ones(batch_size, 3, grid_size, grid_size, dtype=torch.bool, device=predictions.device)
        
        tx = torch.zeros(batch_size, 3, grid_size, grid_size, device=predictions.device)
        ty = torch.zeros(batch_size, 3, grid_size, grid_size, device=predictions.device)
        tw = torch.zeros(batch_size, 3, grid_size, grid_size, device=predictions.device)
        th = torch.zeros(batch_size, 3, grid_size, grid_size, device=predictions.device)
        tconf = torch.zeros(batch_size, 3, grid_size, grid_size, device=predictions.device)
        tcls = torch.zeros(batch_size, 3, grid_size, grid_size, self.num_classes, device=predictions.device)
        
        # Convertir les anchors
        scaled_anchors = torch.FloatTensor([(a_w / stride, a_h / stride) for a_w, a_h in anchors]).to(predictions.device)
        anchor_w = scaled_anchors[:, 0:1].view((1, 3, 1, 1))
        anchor_h = scaled_anchors[:, 1:2].view((1, 3, 1, 1))
        
        # Traiter les cibles
        for b in range(batch_size):
            for target in targets:
                if target.sum() == 0:  # Si pas de cible
                    continue
                    
                # Coordonnées x, y, w, h normalisées à la taille de l'image
                gx = target[1] * grid_size
                gy = target[2] * grid_size
                gw = target[3] * self.img_size
                gh = target[4] * self.img_size
                
                # Indices de la cellule de la grille
                gi = int(gx)
                gj = int(gy)
                
                # Convertir les dimensions relatives à l'image en dimensions relatives à l'anchor
                gw_anchors = gw / stride
                gh_anchors = gh / stride
                
                # Trouver le meilleur anchor (IOU plus grand)
                anchor_ious = []
                for anchor_idx, anchor in enumerate(scaled_anchors):
                    anchor_iou = self.calculate_iou_anchors(gw_anchors, gh_anchors, anchor[0], anchor[1])
                    anchor_ious.append(anchor_iou)
                
                best_anchor = np.argmax(anchor_ious)
                
                # Vérifier si la cible est dans les limites de la grille
                if gi < grid_size and gj < grid_size:
                    # Marquer la cellule comme contenant un objet
                    obj_mask[b, best_anchor, gj, gi] = True
                    noobj_mask[b, best_anchor, gj, gi] = False
                    
                    # Coordonnées relatives à la cellule
                    tx[b, best_anchor, gj, gi] = gx - gi
                    ty[b, best_anchor, gj, gi] = gy - gj
                    
                    # Largeur et hauteur en log-espace
                    tw[b, best_anchor, gj, gi] = torch.log(gw_anchors / scaled_anchors[best_anchor][0] + 1e-16)
                    th[b, best_anchor, gj, gi] = torch.log(gh_anchors / scaled_anchors[best_anchor][1] + 1e-16)
                    
                    # Confiance (objectness)
                    tconf[b, best_anchor, gj, gi] = 1
                    
                    # Classe (one-hot encoding)
                    class_idx = int(target[5])
                    tcls[b, best_anchor, gj, gi, class_idx] = 1
        
        # Calculer les pertes
        loss_x = self.mse(x[obj_mask], tx[obj_mask])
        loss_y = self.mse(y[obj_mask], ty[obj_mask])
        loss_w = self.mse(w[obj_mask], tw[obj_mask])
        loss_h = self.mse(h[obj_mask], th[obj_mask])
        
        loss_conf_obj = self.mse(conf[obj_mask], tconf[obj_mask])
        loss_conf_noobj = self.mse(conf[noobj_mask], tconf[noobj_mask])
        loss_conf = loss_conf_obj + self.lambda_noobj * loss_conf_noobj
        
        loss_cls = self.mse(pred_cls[obj_mask], tcls[obj_mask])
        
        # Perte totale
        loss = (
            self.lambda_coord * (loss_x + loss_y + loss_w + loss_h)
            + loss_conf
            + loss_cls
        )
        
        return loss

    def calculate_iou_anchors(self, target_w, target_h, anchor_w, anchor_h):
        """Calcule l'IoU entre une cible et un anchor"""
        intersection = min(target_w, anchor_w) * min(target_h, anchor_h)
        union = (target_w * target_h) + (anchor_w * anchor_h) - intersection
        return intersection / union

def targets_to_yolo_format(targets, img_size=416, num_classes=20):
    """
    Convertit les cibles du format PyTorch (boîtes xmin, ymin, xmax, ymax) 
    au format YOLO (x_center, y_center, width, height, class)
    """
    yolo_targets = []
    
    for batch_idx, target in enumerate(targets):
        boxes = target['boxes']
        labels = target['labels']
        
        # Initialiser un tableau pour ce lot
        batch_targets = torch.zeros((len(boxes), 6))  # 6 = [batch_idx, x, y, w, h, class]
        
        if len(boxes) > 0:
            # Normaliser à la taille de l'image
            boxes_norm = boxes.clone()
            
            # x_center, y_center, width, height
            boxes_norm[:, 0] = ((boxes[:, 0] + boxes[:, 2]) / 2) / img_size
            boxes_norm[:, 1] = ((boxes[:, 1] + boxes[:, 3]) / 2) / img_size
            boxes_norm[:, 2] = (boxes[:, 2] - boxes[:, 0]) / img_size
            boxes_norm[:, 3] = (boxes[:, 3] - boxes[:, 1]) / img_size
            
            # Ajuster les labels pour correspondre à l'indexation YOLO
            # Dans PASCAL VOC, 'background' est la classe 0, mais pour YOLO, on l'ignore
            # donc on soustrait 1 (si nécessaire)
            labels_norm = labels - 1  # Si 'background' est 0 dans vos labels
            
            batch_targets[:, 0] = batch_idx
            batch_targets[:, 1:5] = boxes_norm
            batch_targets[:, 5] = labels_norm
        
        yolo_targets.append(batch_targets)
    
    return torch.cat(yolo_targets, 0)

def train_one_epoch(model, dataloader, optimizer, loss_fn, device, anchors):
    """Entraîne le modèle pour une époque"""
    model.train()
    total_loss = 0
    
    progress_bar = tqdm(dataloader, desc=f"Training")
    for batch_idx, (images, targets) in enumerate(progress_bar):
        images = [img.to(device) for img in images]
        
        # Convertir les cibles au format YOLO
        yolo_targets = targets_to_yolo_format(targets, img_size=IMG_SIZE)
        yolo_targets = yolo_targets.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(images)
        
        # Calculer la perte pour chaque échelle
        loss = torch.tensor(0.0, device=device)
        for i, output in enumerate(outputs):
            scale_loss = loss_fn(output, yolo_targets, anchors[i])
            loss += scale_loss
        
        # Backward pass et optimisation
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        # Mettre à jour la barre de progression
        progress_bar.set_postfix(loss=loss.item())
    
    return total_loss / len(dataloader)

def validate_minimal(model, dataloader, device):
    """Version très simplifiée de validation qui évite tout traitement complexe"""
    model.eval()
    
    # Simplement faire un forward pass sur une seule batch sans calcul de perte
    try:
        with torch.no_grad():
            for images, _ in dataloader:
                images = [img.to(device) for img in images]
                _ = model(images)  # Juste faire un forward pass
                break  # Une seule batch suffit pour vérifier
        return 999.0  # Valeur fictive de perte
    except Exception as e:
        print(f"Validation error (ignored): {e}")
        return 999.0  # Valeur fictive en cas d'erreur

def train(model, train_loader, val_loader, device, epochs=10):
    """Fonction principale d'entraînement"""
    # Créer le dossier pour les checkpoints
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    
    # Définir l'optimiseur
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    
    # Définir le scheduler pour réduire le LR
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.1, patience=3, verbose=True
    )
    
    # Définir la fonction de perte pour chaque échelle
    loss_function = YOLOLoss(ANCHORS, NUM_CLASSES, img_size=IMG_SIZE)
    
    # Historique des pertes
    train_losses = []
    val_losses = []
    
    # Meilleure perte de validation
    best_val_loss = float('inf')
    
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        
        # Entraîner pour une époque
        train_loss = train_one_epoch(model, train_loader, optimizer, loss_function, device, anchors)
        train_losses.append(train_loss)
        
        # Utiliser la validation simplifiée
        val_loss = validate_minimal(model, val_loader, device)  # Passer device explicitement
        print(f"Validation simplifiée: val_loss = {val_loss:.6f}")
        val_losses.append(val_loss)
        
        # Mettre à jour le scheduler
        scheduler.step(val_loss)
        
        print(f"Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
        
        # Sauvegarder le meilleur modèle
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            checkpoint = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
            }
            torch.save(checkpoint, os.path.join(CHECKPOINT_DIR, 'best_model.pth'))
            print(f"Saved best model with validation loss: {val_loss:.6f}")
        
        # Sauvegarder checkpoint
        if (epoch + 1) % 5 == 0 or (epoch + 1) == epochs:
            checkpoint = {
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'train_loss': train_loss,
                'val_loss': val_loss,
            }
            torch.save(checkpoint, os.path.join(CHECKPOINT_DIR, f'checkpoint_epoch_{epoch+1}.pth'))
    
    # Tracer les courbes de perte
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.savefig(os.path.join(CHECKPOINT_DIR, 'loss_curve.png'))
    plt.show()
    
    return train_losses, val_losses

def visualize_detections(image, detections, class_names, conf_thresh=0.5):
    """
    Visualise les détections sur une image
    
    Args:
        image: Tensor d'image [C, H, W]
        detections: Liste de détections après NMS
        class_names: Liste des noms de classes
        conf_thresh: Seuil de confiance minimum à afficher
    """
    # Convertir le tensor en image PIL pour l'affichage
    img_np = image.permute(1, 2, 0).cpu().numpy()
    # Dénormaliser
    img_np = img_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img_np = np.clip(img_np, 0, 1)
    
    # Créer la figure
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(img_np)
    
    # Couleurs pour différentes classes
    colors = plt.cm.hsv(np.linspace(0, 1, len(class_names)))
    
    # Si des détections existent
    if detections is not None:
        for x1, y1, x2, y2, obj_conf, cls_conf, cls_id in detections:
            if obj_conf * cls_conf < conf_thresh:
                continue
                
            # Coordonnées de la boîte
            box_h = y2 - y1
            box_w = x2 - x1
            
            # Créer un rectangle
            color = colors[int(cls_id) % len(colors)]
            bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2, 
                                     edgecolor=color, facecolor="none")
            ax.add_patch(bbox)
            
            # Ajouter le texte avec la classe et la confiance
            class_name = class_names[int(cls_id)]
            conf = obj_conf * cls_conf
            ax.text(x1, y1, f'{class_name}: {conf:.2f}', 
                    color='white', fontsize=10,
                    bbox=dict(facecolor=color, alpha=0.7))
    
    plt.axis("off")
    plt.tight_layout()
    plt.show()
    return fig

###########################################
# PARTIE 5: PROGRAMME PRINCIPAL #
###########################################

if __name__ == "__main__":
    # Configuration
    BATCH_SIZE = 4
    LEARNING_RATE = 0.001
    WEIGHT_DECAY = 0.0005
    EPOCHS = 10
    NUM_CLASSES = 20
    IMG_SIZE = 416
    
    # Vérifier si CUDA est disponible
    device = torch.device("cpu")
    print(f"Utilisation de: {device}")
    
    # Préparer les jeux de données
    print("Préparation des jeux de données...")
    train_loader, val_loader = prepare_data_loaders(BATCH_SIZE, MAX_SAMPLES)
    
    # Afficher quelques statistiques
    print(f"Jeu d'entraînement: {len(train_loader.dataset)} images")
    print(f"Jeu de validation: {len(val_loader.dataset)} images")
    
    # Initialiser le modèle
    model = YOLOv3(NUM_CLASSES).to(device)
    
    # Choix de l'action à effectuer
    print("\nOptions disponibles:")
    print("1. Entraîner le modèle")
    print("2. Charger un modèle pré-entraîné")
    print("3. Visualiser les prédictions du modèle par défaut")
    
    choice = input("Votre choix (1/2/3): ")
    
    if choice == "1":
        # Entraîner le modèle
        print("\nDémarrage de l'entraînement...")
        train_losses, val_losses = train(model, train_loader, val_loader, device, EPOCHS)
        print("Entraînement terminé!")
    
    elif choice == "2":
        # Charger un modèle pré-entraîné
        model_path = input("Chemin du modèle (ex: ./checkpoints/best_model.pth): ")
        try:
            checkpoint = torch.load(model_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            print(f"Modèle chargé de l'époque {checkpoint['epoch']} avec perte de validation: {checkpoint['val_loss']:.6f}")
        except Exception as e:
            print(f"Erreur lors du chargement du modèle: {e}")
            print("Utilisation du modèle par défaut...")
    
    # Passer en mode évaluation
    model.eval()
    
    # Créer un dataloader spécifique pour la visualisation
    test_loader = DataLoader(
        val_loader.dataset, 
        batch_size=1,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=0
    )
    
    # Visualiser quelques exemples
    print("\nVisualisation des prédictions...")
    try:
        num_examples_input = input("Nombre d'exemples à visualiser (1-5) [défaut: 1]: ")
        num_examples = int(num_examples_input) if num_examples_input.strip() else 1
    except ValueError:
        print("Entrée invalide, utilisation de la valeur par défaut (1)")
        num_examples = 1
    num_examples = max(1, min(5, num_examples))
    
    try:
        conf_threshold_input = input("Seuil de confiance (0.1-0.9) [défaut: 0.5]: ")
        conf_threshold = float(conf_threshold_input) if conf_threshold_input.strip() else 0.5
    except ValueError:
        print("Entrée invalide, utilisation de la valeur par défaut (0.5)")
        conf_threshold = 0.5
    conf_threshold = max(0.1, min(0.9, conf_threshold))
    
    nms_threshold = 0.4  # Valeur standard pour NMS
    
    # Visualiser les prédictions
    for i, (images, targets) in enumerate(test_loader):
        if i >= num_examples:
            break
            
        image = images[0].to(device)
        target = targets[0]
        
        # Obtenir les prédictions
        with torch.no_grad():
            predictions = model([image])
        
        # Post-traitement pour obtenir les détections
        processed_preds = []
        for j, pred in enumerate(predictions):
            processed = transform_predictions(pred, IMG_SIZE, ANCHORS[j], NUM_CLASSES, device)
            processed_preds.append(processed)
        
        # Combiner les prédictions des trois échelles
        detections = torch.cat(processed_preds, 1)
        
        # Appliquer NMS
        output = non_max_suppression(detections, NUM_CLASSES, conf_threshold, nms_threshold)
        
        # Afficher l'image avec les détections
        print(f"\nImage {i+1} - Visualisation des détections:")
        
        # Afficher les boîtes réelles
        img_np = image.cpu()
        img_np = img_np * torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1) + \
                 torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        img_np = torch.clamp(img_np, 0, 1)
        
        # Créer deux sous-figures
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 9))
        
        # Image originale avec annotations réelles
        img_display = img_np.permute(1, 2, 0).cpu().numpy()
        ax1.imshow(img_display)
        ax1.set_title('Annotations réelles')
        ax1.axis('off')
        
        # Dessiner les boîtes réelles
        boxes = target['boxes'].cpu().numpy()
        labels = target['labels'].cpu().numpy()
        
        for box, label in zip(boxes, labels):
            xmin, ymin, xmax, ymax = box
            class_name = VOC_CLASSES[label]
            
            # Mettre à l'échelle pour l'affichage
            xmin = xmin * IMG_SIZE / image.shape[2]
            ymin = ymin * IMG_SIZE / image.shape[1]
            xmax = xmax * IMG_SIZE / image.shape[2]
            ymax = ymax * IMG_SIZE / image.shape[1]
            
            width = xmax - xmin
            height = ymax - ymin
            
            rect = patches.Rectangle((xmin, ymin), width, height, 
                                    linewidth=2, edgecolor='green', facecolor='none')
            ax1.add_patch(rect)
            ax1.text(xmin, ymin, class_name, color='white', fontsize=10,
                     bbox=dict(facecolor='green', alpha=0.7))
        
        # Image avec détections prédites
        ax2.imshow(img_display)
        ax2.set_title('Détections prédites')
        ax2.axis('off')
        
        # Dessiner les boîtes prédites
        if output[0] is not None:
            for box in output[0]:
                x1, y1, x2, y2, conf, cls_conf, cls_pred = box
                
                # Mettre à l'échelle
                x1 = x1.item()
                y1 = y1.item()
                x2 = x2.item()
                y2 = y2.item()
                
                width = x2 - x1
                height = y2 - y1
                
                class_idx = int(cls_pred.item())
                class_name = VOC_CLASSES[class_idx + 1]  # +1 car nous avons ignoré 'background'
                
                rect = patches.Rectangle((x1, y1), width, height, 
                                        linewidth=2, edgecolor='red', facecolor='none')
                ax2.add_patch(rect)
                ax2.text(x1, y1, f"{class_name}: {conf.item()*cls_conf.item():.2f}", 
                         color='white', fontsize=10,
                         bbox=dict(facecolor='red', alpha=0.7))
        
        plt.tight_layout()
        plt.show()
        
        # Pause pour voir les résultats
        if i < num_examples - 1:
            input("Appuyez sur Entrée pour voir l'exemple suivant...")
    
    print("\nVisualisation terminée!")
    print("Merci d'avoir utilisé le modèle YOLOv3 pour la détection d'objets!")

Utilisation de: cpu
Préparation des jeux de données...
Jeu d'entraînement: 3 images
Jeu de validation: 1 images

Options disponibles:
1. Entraîner le modèle
2. Charger un modèle pré-entraîné
3. Visualiser les prédictions du modèle par défaut

Visualisation des prédictions...
